## Non-Negative Matrix Factorization for Exploring the European Parliament's Topic Agenda
### Assignment 2 for Machine Learning Complements class
By Alexandra de Carvalho, Luís Costa, Nuno Pedrosa

#### Importing the needed Python libraries
We will use Pandas for dataframe manipulation.

In [2]:
#!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [3]:
import os
import re
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import pandas as pd

# for modeling 
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF

# for text processing
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim import corpora, models
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nunop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nunop\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nunop\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nunop\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Static NMF

#### Importing the data

The archive sample.zip contains a sample corpus of 1,324 news articles divided into three time windows (month1, month2, month3).

To run the code, please unzip the data.

In [4]:
# expand pandas df column display width to enable easy inspection
pd.set_option('max_colwidth', 150)

# read the textfiles to a dataframe
dir_path = 'sample' # folder path
files = [] # list to store files

for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(os.path.join(dir_path, path))
    else:
        subpath = os.path.join(dir_path, path)
        for path2 in os.listdir(subpath):
            if os.path.isfile(os.path.join(subpath, path2)):
                files.append(os.path.join(subpath, path2))

#### Tokenizing
To make all of the text in the speeches as comparable as possible we need to remove punctuation, capitalization, numbers, and strange characters. We also keep the term frequency on each document.

With the before in mind, we will create text_tokens, that will be a dictionary of dictionarys. It is structured in such a way that, for each month and type sample key, we have a dictionary corresponding to the tokenized words with their count in the respective sample.

In [5]:
text_tokens = dict()
for filename in files:
    with open(filename, 'rb') as f:
        lines = f.readlines()
        text_tokens[filename] = dict()
        
        for line in lines:
            for token in re.split('\W+', str(line)):
                token = token.lower()
                if len(token) > 3 and not token.isnumeric() and not token.lower() in stopwords.words('english'):
                    text_tokens[filename][token] = text_tokens[filename].get(token, 0) + 1

#### Lemmatizing

Another important step is to lemmatize the words, that is to convert verbs, adjetives, and others variations of the same word into their base form. This allows us to analyse related words as a single one and reduces the number of words in the documents matrix, reducing sparcity.

Ex: walked -> walk

With the before in mind, we will create nouns, that will be a dictionary of dictionarys. It is structured in a similar way as text_tokens, but with the count by lemmatized word.

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()   # stored function to lemmatize each word
is_noun = lambda pos: pos[:2] == 'NN'

nouns = dict()
for filename, tokens in text_tokens.items():
    if filename not in nouns:
        nouns[filename] = dict()

    for (word, pos) in pos_tag(list(tokens.keys())):
        if is_noun(pos):
            nouns[filename][wordnet_lemmatizer.lemmatize(word)] = nouns[filename].get(wordnet_lemmatizer.lemmatize(word), 0) + text_tokens[filename][word]

#### Building the matrix A

A is a matrix where every line corresponds to a specific document, and every column corresponds to a specific term/token, in this way, we get a matrix with the menbership of each term in each document. In a initial state, these menberships will be represented with it's term frequency weights.

In [10]:
dictvectorizer = DictVectorizer(sparse=False)
a = dictvectorizer.fit_transform(list(nouns.values()))

In [14]:
# 1324 documents and 11236 different terms
a.shape

(1324, 11236)

Building the list of all tokens (all columns of A, in order).

In [12]:
token_list = dictvectorizer.get_feature_names()

TF-IDF: (term frequency–inverse document frequency) term weighting  

The tf–idf value of a word increases proportionally as the number of occurrences of it in a document increases, however, this value is balanced by the normal frequency of that word. This helps to distinguish the fact that the occurrence of some words is generally more common than others.

This also helps to produce diverse but semantically coherent topics.

Now calculating the TF-IDF weights and updating the A matrix with them.

In [17]:
for column_idx in range(len(token_list)):
    idf = math.log(len(a[:, column_idx])/len([x for x in a[:, column_idx] if x != 0]), 10)

    for element_idx in range(len(files)):
        if a[element_idx,column_idx] != 0:
            a[element_idx,column_idx] = (math.log(a[element_idx,column_idx], 10) + 1) * idf

#### Finding the best value for K : TC-W2V

We will use the same number of terms per topic *t* as the paper (10).

In [21]:
# number of terms per topic
t = 10

To find the best number of topics k, we will use a topic coherence measure called TC-W2V. In this approach, the coherence of each topic is measured using the cosine similarity. The coherence of a model is the mean coherence of the topics of the model.

We run the NMF with a number of topics that ranges between 10 and 25 in order to find the ammount of topics that gives us a model with the biggest coherence.

In [52]:
max_model_coherence = 0
res_k = 0

for k in range(10,26):

    nmf_model = NMF(k, random_state=1) 
    nmf_model.fit_transform(a)

    vocabulary = [[token_list[x[1]] for x in sorted(zip(topic,range(len(topic))), reverse = True)[:t]] for topic in nmf_model.components_]
    model = Word2Vec(sentences = vocabulary, vector_size = 200, window = 5, hs = 1, negative = 0, min_count = 1)
    
    # calculating individual topic coherence scores for each topic
    model_score = []
    for topic in vocabulary:
        topic_score = []
        for w1 in topic:
            for w2 in topic:
                if w2 > w1:
                    word_score = cosine_similarity(model.wv[w2].reshape(1,-1),model.wv[w1].reshape(1,-1))[0]
                    topic_score.append(word_score[0])
        
        topic_score = sum(topic_score)/len(topic_score) # mean of each word pair similarity in the topic
        model_score.append(topic_score)

    model_coherence = sum(model_score)/len(model_score) # mean of topic coherence in the model
    print("k = ",k, ". Model coherence:", model_coherence)

    # used in order to choose the number of topics that has the biggest coherence
    if model_coherence > max_model_coherence:
        max_model_coherence = model_coherence
        res_k = k

c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


k =  10 . Model coherence: 0.0023823067576934894


c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


k =  11 . Model coherence: 0.005569110116498036


c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


...

k =  23 . Model coherence: 0.00273876978525361


c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


k =  24 . Model coherence: 0.001925166782112447


c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


k =  25 . Model coherence: 0.0035117933278282485


In [53]:
# res_k is the best number of topics obtained
res_k

11

#### NMF

To obtain topics that suit our data, we will use NMF (Non-negative matrix factorization).

It is a group of algorithms where a matrix A is factored into two matrices, one W and one H, with the property that the matrices have no negative elements. This makes the resulting matrices easier to analyze.

General formula:
V = W * H


The W and H matrices have special features:


- The k lines of H represent the k topics defined by non-negative weights, the columns represent the terms. This gives the rank of the term in the topics: the higher the value, the higher the rank of the word in this topic. 
Sorting a line gives us the description of the topic: a ranking of the most related terms, allowing interpretation of the themes.

- The k columns of W the topics, and the lines represents the documents/speeches. The values represent the weights of the menbership of each document in a topic.

The number of topics that gives us the biggest coherence is 11, so that is the number of topics that we will use in our definitive NMF:

In [54]:
nmf_model = NMF(res_k, random_state=1) 
w = nmf_model.fit_transform(a)

c:\Users\nunop\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


#### Results

Now, we will analyse the constitution of the obtained topics:

For each topic, find the t higher weights index and find the correpondent token (same index) in the token list. These are the descriptors of each topic.

In [55]:
for i, topic in enumerate(nmf_model.components_):
    print("Topic", i, ":",[token_list[x[1]] for x in sorted(zip(topic,range(len(topic))), reverse = True)[:t]])

Topic 0 : ['technology', 'phone', 'video', 'speed', 'generation', 'device', 'network', 'broadband', 'image', 'picture']
Topic 1 : ['club', 'player', 'football', 'team', 'chelsea', 'game', 'season', 'manager', 'champion', 'league']
Topic 2 : ['election', 'blair', 'party', 'minister', 'government', 'leader', 'tory', 'secretary', 'chancellor', 'democrat']
Topic 3 : ['music', 'band', 'song', 'rock', 'artist', 'album', 'singer', 'record', 'single', 'award']
Topic 4 : ['forsyth', 'frederick', 'terrorist', 'internment', 'forsythe', 'totalitarianism', 'qaeda', 'fundamentalism', 'churchill', 'liberty']
Topic 5 : ['growth', 'economy', 'market', 'price', 'rate', 'rise', 'bank', 'investment', 'analyst', 'dollar']
Topic 6 : ['angel', 'rhapsody', 'bland', 'brit', 'guy', 'pulp', 'cheesy', 'deserve', 'joss', 'joke']
Topic 7 : ['sub', 'minute', 'goal', 'ball', 'yard', 'header', 'kick', 'cech', 'duff', 'cross']
Topic 8 : ['software', 'virus', 'user', 'mail', 'program', 'computer', 'security', 'site', 'i

Analysing the obtained topics, we can see that they correspond to a specific theme, showing that they were well obtained.
For example, in topic 3, we have just terms related to music, topic 5 has terms related to economy and topic 10 has terms related to movies. (These results can change if we run the algorithm again.)

Next, we can then observe the documents with bigger weights for each topic. Because the files names already tag the contained speech by topic, we can infer the validity of the model built.

In [56]:
for i in range(res_k):
    print("Topic", i, ":",[files[x[1]].split('/')[-1] for x in sorted(zip(w[:,i],range(len(w[:,i]))), reverse = True)[:t]])

Topic 0 : ['sample\\month1\\tech_032.txt', 'sample\\month3\\tech_335.txt', 'sample\\month3\\tech_294.txt', 'sample\\month2\\tech_155.txt', 'sample\\month3\\tech_396.txt', 'sample\\month1\\tech_009.txt', 'sample\\month3\\tech_309.txt', 'sample\\month3\\tech_313.txt', 'sample\\month1\\tech_094.txt', 'sample\\month3\\tech_216.txt']
Topic 1 : ['sample\\month1\\football_018.txt', 'sample\\month1\\football_027.txt', 'sample\\month3\\football_207.txt', 'sample\\month1\\football_024.txt', 'sample\\month1\\football_088.txt', 'sample\\month1\\football_087.txt', 'sample\\month3\\football_202.txt', 'sample\\month3\\football_223.txt', 'sample\\month2\\football_180.txt', 'sample\\month1\\football_086.txt']
Topic 2 : ['sample\\month3\\politics_253.txt', 'sample\\month3\\politics_218.txt', 'sample\\month3\\politics_255.txt', 'sample\\month3\\politics_252.txt', 'sample\\month3\\politics_256.txt', 'sample\\month3\\politics_254.txt', 'sample\\month3\\politics_260.txt', 'sample\\month3\\politics_251.txt',

As we can see, theres is definitively a match between the terms in each topic and the name of the speechs from which they were recovered. This results are aligned with what was expected and as such verify that the model built is a valid one.

### LDA implementation

Now, with the objective of comparing our results, we will apply LDA to our data.

In [48]:
# Pre-processing the speeches
text_tokens = []
for filename in files:
    with open(filename, 'rb') as f:
        lines = f.readlines()
        sup_list = []
        for line in lines:
            for token in re.split('\W+', str(line)):
                token = token.lower()
                if len(token) > 3 and not token.isnumeric() and not token.lower() in stopwords.words('english'):
                    sup_list.append(token)
    text_tokens.append(sup_list)

for doc in text_tokens:
    doc = [wordnet_lemmatizer.lemmatize(x) for x in doc]

# only used if the file is run on macOs due to the .DS_Store file
# text_tokens.pop(0)

In [57]:
# Turn the tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(text_tokens)

# Convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in text_tokens]

# We will use the same number of topics as in the NMF so we can compare
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=11, id2word=dictionary, passes=20)

# Printing the results
for idx, topic in ldamodel.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic} \n")


Topic: 0 
Words: 0.014*"said" + 0.012*"people" + 0.006*"could" + 0.005*"games" + 0.005*"technology" + 0.005*"users" + 0.004*"microsoft" + 0.004*"also" + 0.004*"online" + 0.004*"software" 

Topic: 1 
Words: 0.022*"said" + 0.013*"would" + 0.011*"government" + 0.009*"labour" + 0.008*"election" + 0.008*"blair" + 0.008*"party" + 0.006*"people" + 0.005*"also" + 0.005*"howard" 

Topic: 2 
Words: 0.011*"said" + 0.007*"game" + 0.007*"would" + 0.007*"club" + 0.006*"chelsea" + 0.006*"united" + 0.006*"players" + 0.005*"arsenal" + 0.005*"league" + 0.005*"time" 

Topic: 3 
Words: 0.013*"best" + 0.011*"film" + 0.011*"music" + 0.008*"said" + 0.008*"show" + 0.007*"year" + 0.006*"also" + 0.006*"band" + 0.006*"awards" + 0.005*"award" 

Topic: 4 
Words: 0.018*"said" + 0.009*"mail" + 0.007*"people" + 0.007*"spam" + 0.006*"virus" + 0.005*"site" + 0.005*"also" + 0.005*"many" + 0.004*"sites" + 0.004*"attacks" 

Topic: 5 
Words: 0.028*"said" + 0.007*"would" + 0.005*"people" + 0.004*"police" + 0.004*"rights" + 

As we can see from the results, the LDA model returns terms such as "would", "said" and "also" which are speech conectors and verbs that not really related to the topics produced. . As such we can conclude that the NMF approach produces more fine grained results, being able to move past the noise present in the text and to really capture the most common terms per topic.